In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install -Uqq fastai

In [2]:
from fastai.vision.all import *
import fastai

In [9]:
path = untar_data(URLs.IMAGENETTE)

In [10]:
path

Path('/root/.fastai/data/imagenette2')

In [11]:
dblock = DataBlock(
    blocks = (ImageBlock(),CategoryBlock()),
    get_items= get_image_files,
    get_y = parent_label,
    item_tfms= Resize(460),
    batch_tfms= aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path,bs=64)

In [7]:
model = xresnet50(n_out = dls.c)
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.621904,1.720744,0.515683,03:28
1,1.262739,1.454742,0.537341,03:23
2,1.000755,1.359159,0.592233,03:26
3,0.780303,0.762987,0.758402,03:26
4,0.601230,0.594197,0.817774,03:24


In [8]:
x,y = dls.one_batch()

In [11]:
x.mean(dim = [0,2,3]),x.std(dim= [0,2,3])

(TensorImage([0.4699, 0.4573, 0.4343], device='cuda:0'),
 TensorImage([0.2718, 0.2688, 0.2951], device='cuda:0'))

In [12]:
def get_dls(bs, size):
    dblock = DataBlock(
        blocks = (ImageBlock,CategoryBlock),
        get_items = get_image_files,
        get_y = parent_label,
        item_tfms = Resize(460),
        batch_tfms = [*aug_transforms(size = size, min_scale= 0.75),
                     Normalize.from_stats(*imagenet_stats)]
    )
    return dblock.dataloaders(path,bs = bs)

In [13]:
dls = get_dls(64,224)

In [27]:
x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([-0.1394, -0.0720,  0.0026], device='cuda:0'),
 TensorImage([1.2576, 1.2389, 1.3132], device='cuda:0'))

In [29]:
model = xresnet50(n_out = dls.c)
learn = Learner(dls, model, loss_func = CrossEntropyLossFlat(), metrics = accuracy)
learn.fit_one_cycle(5,3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.600239,3.069070,0.376400,03:26
1,1.241070,1.948895,0.460792,03:19
2,0.961104,1.034894,0.661314,03:19
3,0.770451,0.668365,0.792382,03:24
4,0.597892,0.595286,0.809933,03:30


In [15]:
dls = get_dls(128,128)
learn = Learner(dls, xresnet18(n_out = dls.c), loss_func = CrossEntropyLossFlat(),
               metrics = accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.576410,1.792295,0.466766,02:54
1,1.140719,1.026570,0.677371,02:49
2,0.870712,0.846010,0.713592,02:51
3,0.694429,0.654162,0.796490,02:54


In [16]:
learn.dls = get_dls(64, 224)
learn.fine_tune(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.777315,0.798687,0.752801,02:55


epoch,train_loss,valid_loss,accuracy,time
0,0.631312,0.699177,0.780060,02:59
1,0.594227,0.646116,0.797237,02:59
2,0.537688,0.517161,0.834951,02:58
3,0.458123,0.435055,0.867438,03:00
4,0.394365,0.406773,0.875280,02:57
